In [5]:
import seaborn as sns
import numpy as np
import pandas as pd

### 1. IRIS

#### a. 기초통계량

In [6]:
iris = sns.load_dataset("iris")

In [14]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


#### b.이상치를 제거한 평균과 표준편차

In [243]:
# sepal_length 이상치제거
Q1 = iris['sepal_length'].quantile(0.25)
Q3 = iris['sepal_length'].quantile(0.75)
IQR = Q3 - Q1

outer_sl = iris[(iris['sepal_length']<Q3+IQR*1.5) & (iris['sepal_length']>Q1-IQR*1.5)]
outer_sl

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [247]:
# sepal_width 이상치제거
Q1 = outer_sl['sepal_width'].quantile(0.25)
Q3 = outer_sl['sepal_width'].quantile(0.75)
IQR = Q3 - Q1

outer_sw = outer_sl[(outer_sl['sepal_width']<Q3+IQR*1.5) & (outer_sl['sepal_width']>Q1-IQR*1.5)]
outer_sw

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [248]:
# petal_length 이상치제거
Q1 = outer_sw['petal_length'].quantile(0.25)
Q3 = outer_sw['petal_length'].quantile(0.75)
IQR = Q3 - Q1

outer_pl = outer_sw[(outer_sw['petal_length']<Q3+IQR*1.5) & (outer_sw['petal_length']>Q1-IQR*1.5)]
outer_pl

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [250]:
# petal_width 이상치제거
Q1 = outer_pl['petal_width'].quantile(0.25)
Q3 = outer_pl['petal_width'].quantile(0.75)
IQR = Q3 - Q1

outer_pw = outer_pl[(outer_pl['petal_width']<Q3+IQR*1.5) & (outer_pl['petal_width']>Q1-IQR*1.5)]
outer_pw

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [252]:
# 4가지 항목의 표준편차
round(outer_pw.std(),2)

sepal_length    0.83
sepal_width     0.40
petal_length    1.76
petal_width     0.76
dtype: float64

In [255]:
# 4가지 항목의 평균
round(outer_pw.mean(),2)

sepal_length    5.86
sepal_width     3.04
petal_length    3.81
petal_width     1.22
dtype: float64

### 2.Titanic

#### a. [category1]

In [44]:
titanic = sns.load_dataset('titanic')

In [93]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699094,0.523008,0.381594,32.204208
std,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,29.699000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [45]:
titanic["category1"] = titanic.apply(lambda r: r.sex if r.age>=20 else "child", axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


#### b. ['age']바꾸기

In [46]:
titanic['age'] = round(titanic['age'].fillna(titanic['age'].mean()),3)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.699,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


#### c. 생존률 구하기

In [156]:
# 성별 생존률
sex_pct = round(titanic.groupby("sex").size().astype(int)/len(titanic),2)
sex_pct

sex
female    0.35
male      0.65
dtype: float64

In [157]:
# 선실별 생존률
class_pct = round(titanic.groupby("class").size().astype(int)/len(titanic),2)
class_pct

class
First     0.24
Second    0.21
Third     0.55
dtype: float64

In [158]:
# 출발지별 생존률
embark_town_pct = round(titanic.groupby("embark_town").size().astype(int)/len(titanic),2)
embark_town_pct

embark_town
Cherbourg      0.19
Queenstown     0.09
Southampton    0.72
dtype: float64

#### d. 그룹별 생존률

In [159]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
ages = pd.cut(titanic["age"], bins, labels=labels)
ages_pct = round(titanic.groupby(ages).size().astype(int)/len(titanic),2)
ages_pct

age
미성년자    0.19
청년      0.46
중년      0.27
장년      0.07
노년      0.01
dtype: float64

#### e. qcut을 사용한 그룹별 생존률

In [73]:
titanic["qcut_ages2"] = pd.qcut(titanic["age"], 3, labels=['청소년', '성인', '노인'])
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,qcut_ages2
886,0,2,male,27.000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,성인
887,1,1,female,19.000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,청소년
888,0,3,female,29.699,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child,성인
889,1,1,male,26.000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,성인
890,0,3,male,32.000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,노인


In [87]:
# 나이 그룹별 남녀 성비
round(titanic.groupby(["qcut_ages2", "sex"]).size()/titanic.groupby(["qcut_ages2"]).size(),2)

qcut_ages2  sex   
청소년         female    0.41
            male      0.59
성인          female    0.32
            male      0.68
노인          female    0.33
            male      0.67
dtype: float64

In [88]:
# 나이 그룹별 남녀 생존률
round(titanic.groupby(["qcut_ages2", "alive"]).size()/titanic.groupby(["qcut_ages2"]).size(),2)

qcut_ages2  alive
청소년         no       0.59
            yes      0.41
성인          no       0.66
            yes      0.34
노인          no       0.59
            yes      0.41
dtype: float64

### mpg

In [123]:
mpg = sns.load_dataset('mpg')

#### a. 배기량 대비 마력

In [124]:
mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [125]:
mpg["hp_per_cc"] = round(mpg.displacement / mpg.horsepower,2)
mpg.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,1.63
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,1.87
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,1.61
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,1.52
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy s-10,1.45


#### b. 제조사와 모델을 추출하여 name 컬럼 바꾸기

In [126]:
name_split = mpg['name'].str.split(' ')

manufacturer = name_split.str[0]
model1 = name_split.str[1]
model2 = name_split.str[2]
model = model1+' '+model2

mpg['manufacturer'] = manufacturer
mpg['model'] = model

mpg.drop(['name'], axis='columns', inplace=True)

mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,2.36,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,2.12,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,2.12,plymouth,NaN
3,16.0,8,304.0,150.0,3433,12.0,70,usa,2.03,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,2.16,ford,NaN
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,1.63,ford,mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,1.87,vw,NaN
395,32.0,4,135.0,84.0,2295,11.6,82,usa,1.61,dodge,NaN
396,28.0,4,120.0,79.0,2625,18.6,82,usa,1.52,ford,NaN


#### c. 실린더 갯수별 연비의 평균

In [129]:
round(mpg.groupby(mpg.cylinders)["mpg"].mean(),2)

cylinders
3    20.55
4    29.29
5    27.37
6    19.99
8    14.96
Name: mpg, dtype: float64

#### d. 생산지별 배기량 대비 마력 평균

In [133]:
round(mpg.groupby(mpg.origin)["hp_per_cc"].mean(),2)

origin
europe    1.41
japan     1.30
usa       2.05
Name: hp_per_cc, dtype: float64

#### 모델이 5개 이상인 제조사중 연비 평균이 좋은 제조사 5개

In [208]:
# 모델이 5개인 회사
def is_five(x):
    if x >= 5:
        return x
five = mpg.groupby(mpg.manufacturer).size().astype(int).agg(is_five)
five.sort_values(ascending=False)[:5]

manufacturer
ford         51.0
chevrolet    43.0
plymouth     31.0
amc          28.0
dodge        28.0
dtype: float64